In [2]:
# imports

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import os
from pickle import dump, load
import numpy as np
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import argparse

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

In [3]:
# extract features from images using Xception transfer learning

def get_img_features(filepath):

    xfer_model = Xception(include_top=False, pooling='avg')
    features = {}

    for dir in tqdm(os.listdir(filepath)):
        subfolder = str(filepath + '/' + dir)
        for img in tqdm(os.listdir(subfolder)):
        print(type(img))  
        img_name = subfolder + '/' + img
        #print(img_name)
        image = Image.open(img_name)
        image = image.resize((299,299))
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image -= 1.0

        feature = xfer_model.predict(image)
        features[img] = feature
    
    return features

# transfer learning techniques adapted from neptune.ai's Derrick Mwiti (https://neptune.ai/blog/transfer-learning-guide-examples-for-images-and-text-in-keras)

In [4]:
#img_dirs = []

#root = '/content/drive/MyDrive/vizwiz/train'

#for i in range(1,17):
#  img_dirs.append(root + str(i))

In [5]:
#img_dirs

In [6]:
# notebook originally written on Colab

root_path = '/content/drive/MyDrive/vizwiz'

In [7]:
#data_paths = [os.path.join(root_path, f) for f in os.listdir(root_path)]
#data_paths = [i for i in data_paths if os.path.isfile(i)]

In [83]:
#features = get_img_features(root_path)

In [84]:
#len(features)

In [85]:
#dump(features, open('features.p', 'wb'))

In [86]:
#features = load(open('/content/drive/MyDrive/features.p', 'rb'))

In [8]:
# load img_names.txt

def get_image_names(filepath):
    file = load_doc(filepath)
    imgs = file.split(',')[:-1]

    output = []

    for f in imgs:
        path = ''
        path = f[2:len(f)-1]
        output.append(path)

    return output

In [9]:
#load the clean descriptions

def get_clean_descriptions(filepath, images):
    #load the cleaned up descriptions
    file = load_doc(filepath)
    descriptions = {}

    for line in file.split('\n'):
    
        words = line.split()
        if len(words) < 1:
            continue
    
        image, caption = words[0], words[1:]

        if image in images:
            if image not in descriptions:
            descriptions[image] = []

            desc = '<start> ' + " ".join(caption) + ' <end>'
            descriptions[image].append(desc)

    return descriptions

In [10]:
#load the features

def load_features(imgs):
    #load all the features!
    all_features = load(open('/content/drive/MyDrive/features.p', 'rb'))

    #then get only the needed features
    features = {k:all_features[k] for k in imgs}
    return features

In [11]:
img_names_text = '/content/drive/MyDrive/img_names.txt'

# function to load text file into memory

def load_doc(filepath):
    #open a file as read only
    file = open(filepath, 'r')
    text = file.read()
    file.close()
    return text

In [44]:
train_imgs = get_image_names(img_names_text)

In [45]:
train_descriptions = get_clean_descriptions('/content/drive/MyDrive/images_with_associated_captions.txt', train_imgs)

In [46]:
train_features = load_features(train_imgs)

In [12]:
val_img_path = '/content/drive/MyDrive/val'

In [13]:
val_imgs = get_image_names('/content/drive/MyDrive/val_img_names.txt')

In [14]:
val_descriptions = get_clean_descriptions('/content/drive/MyDrive/val_imgs_w_caps.txt', val_imgs)

In [15]:
# extract features from VALIDATION mages using Xception transfer learning

def get_val_img_features(filepath):

    xfer_model = Xception(include_top=False, pooling='avg')
    features = {}

    for img in tqdm(os.listdir(filepath)):
        print(type(img))  
        #img_name = subfolder + '/' + img
        print(img_name)
        #image = Image.open(img)
        #image = image.resize((299,299))
        #image = np.expand_dims(image, axis=0)
        #image = image/127.5
        #image -= 1.0

        #feature = xfer_model.predict(image)
        #features[img] = feature

    return features

In [16]:
val_features = load(open('/content/drive/MyDrive/val_features.p', 'rb'))

In [17]:
len(val_features)

7750

In [21]:
type(val_features)

dict

In [34]:
X_val = [k for k in val_features.keys()]
y_val = [f for f in val_features.values()]

In [33]:
#X_val[0]
y_val[0]

array([[0.10363426, 0.14685044, 0.00564226, ..., 0.        , 0.16573177,
        0.16373302]], dtype=float32)

In [35]:
# convert dict to clean list of captions

def d_to_l(desc):
    corpus = []

    for k in desc.keys():
        [corpus.append(text) for text in desc[k]]

    return corpus

In [41]:
#validation_test = d_to_l(val_descriptions)

In [42]:
#validation_test[1]

In [36]:
def create_tokenizer(desc):
    corpus = d_to_l(desc)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)

    return tokenizer, max(len(text.split()) for text in corpus)

In [47]:
tokenizer = create_tokenizer(train_descriptions)

tokens = tokenizer[0]
longest_caption = tokenizer[1]

In [48]:
#dump(tokens, open('tokens.p', 'wb'))

In [49]:
lex_count = len(tokens.word_index)+1

In [50]:
lex_count

18031

In [51]:
longest_caption

112

In [52]:
# since there are ~23000 images to train on, all with large feature vectors, I'll need a data generator to process batches

In [53]:
# create i/o sequence pairs from the iamge description

def data_generator(captions, features, tokenizer, longest):
    while 1:
        for k, cap_list in captions.items():
            #retrieve photo features
            feature = features[k][0]
            img_in, seq, word_out = make_seq(tokenizer, longest, cap_list, feature)

            yield [[img_in, seq], word_out]
            
# data generator code inspired and guided by Abhishek Sharma (https://medium.com/mlearning-ai/image-captioning-using-deep-learning-with-source-code-easy-explanation-3f2021a63f14)

In [54]:
def make_seq(tokenizer, longest, cap_list, feature):
      X1, X2, y = [], [], []

      #iterate through each caption for a given image
      for cap in cap_list:
          #encode the sequence
          seq = tokenizer.texts_to_sequences([cap])[0]

          #split one sequence into multiple X,y pairs
          for i in range(1, len(seq)):
                #split into i/o pair
                in_seq, out_seq = seq[:i], seq[i]
                #pad input
                in_seq = pad_sequences([in_seq], maxlen=longest)[0]
                #encode output
                out_seq = to_categorical([out_seq], num_classes=lex_count)[0]

                #store
                X1.append(feature)
                X2.append(in_seq)
                y.append(out_seq)

    return np.array(X1), np.array(X2), np.array(y) 

# code to make sequences specifically for image captioning purposes inspired and guided by Abhishek Sharma (https://medium.com/mlearning-ai/image-captioning-using-deep-learning-with-source-code-easy-explanation-3f2021a63f14)

In [55]:
[a,b],c = next(data_generator(train_descriptions, train_features, tokens, longest_caption))

In [56]:
a.shape

(57, 2048)

In [57]:
b.shape

(57, 112)

In [58]:
c.shape

(57, 18031)

In [59]:
# captioning model

def setup(lex_count, longest_caption):
  
    #compress number of features from cnn
    cnn_feats = Input(shape=(2048,))
    features_1 = Dropout(0.5)(cnn_feats)
    features_2 = Dense(256, activation='relu')(features_1)

    #lstm sequence model
    lstm_in = Input(shape=(longest_caption,))
    sequences_1 = Embedding(lex_count, 256, mask_zero=True)(lstm_in)
    sequences_2 = Dropout(0.5)(se1)
    seequences_3 = LSTM(256)(sequecnes_2)

    #merge the models
    decoder_1 = add([features_2, sequences_3])
    decoder_2 = Dense(256, activation='relu')(decoder_1)
    outputs = Dense(lex_count, activation='softmax')(decoder_2)

    #model it all
    model = Model(inputs=[cnn_feats, lstm_in], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[])

    #summarize
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)

    return model

# model architecture 

In [60]:
# TRAIN IT, FINALLY

print(f'The training dataset has {len(train_imgs)} images in it.')
print(f'There are {len(train_descriptions)} associatedcaptions; 5 for each image.')
print(f'{len(train_features)} features were extracted.')
print(f'There are {lex_count} unique words.')
print(f'The longest caption is {longest_caption} characters long.')

model = setup(lex_count, longest_caption)
epochs = 30
steps = (len(train_descriptions))*0.2

The training dataset has 23430 images in it.
There are 23430 associatedcaptions; 5 for each image.
23430 features were extracted.
There are 18031 unique words.
The longest caption is 112 characters long.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 112)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 112, 256)     4615936     ['input_2[0][0]']                
                                                                                        

In [ ]:
os.mkdir('models')
for i in range(epochs,epochs+1):
    generator = data_generator(train_descriptions, train_features, tokens, longest_caption)
    model.fit(generator, epochs=1, validation_data=(X_val, y_val), steps_per_epoch=steps, verbose=1)
    model.save('models/model_' + str(i) + '.h5')

4686/4686 [==============================] - ETA: 0s - loss: 4.4024

ValueError: ignored

In [ ]:
def get_img_features_vgg16(filepath):

  xfer_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
  features = {}

  for dir in tqdm(os.listdir(filepath)):
    subfolder = str(filepath + '/' + dir)
    for img in tqdm(os.listdir(subfolder)):
      #print(type(img))  
      img_name = subfolder + '/' + img
      #print(img_name)
      image = Image.open(img_name)
      image = image.resize((299,299))
      image = np.expand_dims(image, axis=0)
      image = image/127.5
      image -= 1.0

      feature = xfer_model.predict(image)
      features[img] = feature
    
  return features

In [ ]:
#vgg_features = get_img_features_vgg16(root_path)

In [ ]:
#dump(vgg_features, open('vgg_features.p', 'wb'))

In [ ]:
#vgg_features = load(open('/content/drive/MyDrive/vgg_features.p', 'rb'))

In [ ]:
def load_vgg_features(imgs):
  #load all the features!
  all_features = load(open('/content/drive/MyDrive/vgg_features.p', 'rb'))

  features = {k:all_features[k] for k in imgs}
  return features

In [ ]:
vgg_imgs = get_image_names(img_names_text)

In [ ]:
vgg_descriptions = get_clean_descriptions('/content/drive/MyDrive/images_with_associated_captions.txt', vgg_imgs)

In [ ]:
vgg_features = load_vgg_features(vgg_imgs)

In [ ]:
vgg_tokenizer = create_tokenizer(vgg_descriptions)

vgg_tokens = vgg_tokenizer[0]
vgg_longest_caption = vgg_tokenizer[1]

In [ ]:
vgg_lex_count = len(vgg_tokens.word_index)+1

In [ ]:
# captioning model with VGG

def setup_for_vgg(vgg_lex_count, vgg_longest_caption):
  
  #compress number of features from cnn
  cnn_feats = Input(shape=(2048,))
  fe1 = Dropout(0.5)(cnn_feats)
  fe2 = Dense(256, activation='relu')(fe1)

  #lstm sequence model
  lstm_in = Input(shape=(vgg_longest_caption,))
  se1 = Embedding(vgg_lex_count, 256, mask_zero=True)(lstm_in)
  se2 = Dropout(0.5)(se1)
  se3 = LSTM(256)(se2)

  #merge the models
  dec1 = add([fe2, se3])
  dec2 = Dense(256, activation='relu')(dec1)
  outputs = Dense(vgg_lex_count, activation='softmax')(dec2)

  #model it all
  model = Model(inputs=[cnn_feats, lstm_in], outputs=outputs)
  model.compile(loss='categorical_crossentropy', optimizer='adam')

  #summarize
  print(model.summary())
  plot_model(model, to_file='model.png', show_shapes=True)

  return model

In [ ]:
# TRAIN IT, FINALLY

print(f'The training dataset has {len(vgg_imgs)} images in it.')
print(f'There are {len(vgg_descriptions)} associatedcaptions; 5 for each image.')
print(f'{len(vgg_features)} features were extracted.')
print(f'There are {vgg_lex_count} unique words.')
print(f'The longest caption is {vgg_longest_caption} characters long.')

model = setup_for_vgg(vgg_lex_count, vgg_longest_caption)
epochs = 10
steps = (len(vgg_descriptions))*0.1

The training dataset has 23430 images in it.
There are 23430 associatedcaptions; 5 for each image.
23430 features were extracted.
There are 18031 unique words.
The longest caption is 112 characters long.
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 112)]        0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 112, 256)     4615936     ['input_10[0][0]']               
                                                                                      

In [ ]:
os.mkdir('models')
for i in range(0,epochs):
  generator = data_generator(vgg_descriptions, vgg_features, vgg_tokens, vgg_longest_caption)
  model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
  model.save('models/vgg_model_' + str(i) + '.h5')

InvalidArgumentError: ignored